In [ ]:
import datasets
from datasets import Sequence
from datasets import ClassLabel

def load_conll_dataset(train_path, dev_path, test_path, token_idx, label_idx):    
    def read_conll_file(file_path, token_idx, label_idx):        
        with open(file_path, "r") as f:
            sentences = [[]]
            for line in f:
                if line.startswith("#"):
                    continue
                
                line = line.strip()
                
                if line:
                    split = line.split('\t')
                    sentences[-1].append((split[token_idx], split[label_idx]))
                
                else:
                    if sentences[-1]:
                        sentences.append([])
            
            if not sentences[-1]:
                sentences.pop()

        # Convert sentences to Hugging Face Dataset format
        dataset = {
            "tokens": [[token for token, label in sentence] for sentence in sentences],
            "target": [[label for token, label in sentence] for sentence in sentences],
        }

        return dataset

    train_dset = read_conll_file(train_path, token_idx, label_idx)
    dev_dset = read_conll_file(dev_path, token_idx, label_idx)
    test_dset = read_conll_file(test_path, token_idx, label_idx)

    # Get all possible labels and cast to ClassLabel
    label_set = set()
    for dset in [train_dset, dev_dset, test_dset]:
        for labels in dset["target"]:
            label_set.update(labels)
    label_names = sorted(list(label_set))
    
    train_dset = datasets.Dataset.from_dict(train_dset)
    train_dset = train_dset.cast_column("target", Sequence(ClassLabel(names=label_names)))

    dev_dset = datasets.Dataset.from_dict(dev_dset)
    dev_dset = dev_dset.cast_column("target", Sequence(ClassLabel(names=label_names)))

    test_dset = datasets.Dataset.from_dict(test_dset)
    test_dset = test_dset.cast_column("target", Sequence(ClassLabel(names=label_names)))
    
    # Convert to Hugging Face DatasetDict format
    dataset = datasets.DatasetDict({
            "train": train_dset,
            "validation": dev_dset,
            "test": test_dset
        })

    return dataset

# raw_dataset = load_conll_dataset("./data/train.conllu", "./data/dev.conllu", "./data/test.conllu", 1, 3)
# print(raw_dataset["train"][0]["tokens"])
# print(raw_dataset["train"][0]["target"])
# print(raw_dataset["train"].features["target"].feature.names)


We should take as input a .json file with the training info. The input files will be in columns separated by \t. In the json file we will specify the 'target' columns. If more than one target column, then we will duplicate the task.

In [ ]:
# from hfmtl.tasks.sequence_classification import SequenceClassification
from hfmtl.tasks.token_classification import TokenClassification
from hfmtl.utils import *
from hfmtl.models import *

from transformers import Trainer, TrainingArguments
import easydict
from frozendict import frozendict
import json

# read train_config.json as easydict
with open("config.json", "r") as f:
    args = easydict.EasyDict(json.load(f))

tasks = []
for task in args.tasks:
    if task.task_type == "token_classification":
        tasks.append(
            TokenClassification(
                dataset = load_conll_dataset(task.train_file, task.eval_file, task.test_file, task.tokens_idx, l_idx),
                name = task.task_name,
                y = ["target"] if len(task.label_idx)==1 else [f"target_{i}" for i in range(len(task.label_idx))],
                tokenizer_kwargs = frozendict(padding="max_length", max_length=args.max_seq_length, truncation=True)
            )
        )
    
    elif task.type == "sequence_classification":
        tasks.append(
            SequenceClassification(
                dataset = load_conll_dataset(task.train_file, task.eval_file, task.test_file, task.tokens_idx, l_idx),
                name = task.name,
                y = ["target"] if len(task.label_idx)==1 else [f"target_{i}" for i in range(len(task.label_idx))],
                tokenizer_kwargs = frozendict(padding="max_length", max_length=args.max_seq_length, truncation=True)
            )
        )


pretrained = 'xlm-roberta-base'
model = MultiTaskModel(pretrained, tasks)

training_args = TrainingArguments(
    output_dir = args.output_dir,
    num_train_epochs = args.num_train_epochs,
    learning_rate = args.learning_rate,
    evaluation_strategy = args.logging_strategy,
    save_strategy = args.logging_strategy,
    weight_decay = args.weight_decay,
)
trainer = MultiTaskTrainer(
    model = model,
    tasks = tasks,
    args = training_args,
    train_dataset = model.train_dataset,
    eval_dataset = model.eval_dataset,
    compute_metrics = None,
    tokenizer = model.tokenizer
)
trainer.train()
